In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

breast_cancer = fetch_ucirepo(id=14)

X = breast_cancer.data.features
y = breast_cancer.data.targets
df = breast_cancer.data.original
variables = breast_cancer.variables['name']

def simpleSample(data):
    testing = data.groupby('Class', group_keys=False).sample(frac=0.2)
    learning = data.drop(testing.index)
    return learning, testing

def fold10Sample(data):
    folds = []
    for i in range(0,10):
        folds.append(data.groupby('Class', group_keys=False).sample(4))
        data = data.drop(folds[i].index)
    return folds


In [2]:
def preprune(df):
    return df


In [3]:
def postprune(df):
    return df


In [4]:
class Node:
    def __init__(self, feature=None, results=None, branches=None):
        self.feature = feature  # Feature to split on
        self.results = results  # Stores class labels if node is a leaf node
        self.branches = branches  # Branch for values that are True for the feature


In [ ]:
def laplaceSmoothing(data, totals):
    l = 1
    additive = [l] * data.shape[0]
    data = data*totals
    for col in data.columns: 
        if 0 in data[col].values:
            totals[col] += l*data.shape[0]
            data[col] = data[col].add(additive)
    
    return data/totals

def creatFrequencyTables(data):
    frequencyTable = {}
    frequencyTable['Class'] = data['Class'].value_counts(dropna=False, ascending=True).to_frame()
    for category in data.columns:
        if category == 'Class':
            continue
        else:
            temp = data[['Class',category]].value_counts(dropna=False)
            frequencyTable[category] = pd.DataFrame({'no-recurrence-events':temp['no-recurrence-events'], 'recurrence-events':temp['recurrence-events']}) / frequencyTable['Class']['count']
            #print(frequencyTable[category].values, any([any(np.isnan(sublist)) for sublist in frequencyTable[category].values]), "\n")
            frequencyTable[category].fillna(0, inplace=True)
            if any([0 in sublist for sublist in frequencyTable[category].values]):
                frequencyTable[category] = laplaceSmoothing(frequencyTable[category], frequencyTable['Class']['count'])

    return frequencyTable

def entropy(a,b):
    return -(a*np.log2(a))-(b*np.log2(b))

def entropy(df):
    frequency = creatFrequencyTables(df)
    entropyTable = {}
    for category in frequency:
        if category == 'Class':
            entropyTable[category] = entropy()




def id3(training, testing):
    return True

creatFrequencyTables(df)


{'Class':                       count
 Class                      
 recurrence-events       102
 no-recurrence-events    214,
 'age':        no-recurrence-events  recurrence-events
 age                                           
 20-29              0.004975                NaN
 30-39              0.104478                NaN
 40-49              0.313433                NaN
 50-59              0.353234                NaN
 60-69              0.199005                NaN
 70-79              0.024876                NaN,
 'menopause':            no-recurrence-events  recurrence-events
 menopause                                         
 premeno                0.507463           0.527473
 ge40                   0.467662           0.384615
 lt40                   0.024876           0.021978,
 'tumor-size':             no-recurrence-events  recurrence-events
 tumor-size                                         
 0-4                     0.034826                NaN
 14-Oct                  0.134328  

In [ ]:
import sys
# m = input("Mode?")
# m = int(m)
m = 0
if m == 0:
    d = preprune(df)
elif m == 1:
    d = postprune(df)
elif m == 2:
    d = postprune(preprune(df))
else:
    sys.exit("Not a valid mode")
    
data, testing = simpleSample(d)
_ , validate = simpleSample(data)

divisor = validate.shape[0]

validate['result'] = id3(data, validate)
success = validate[validate['result'] == validate['Class']].shape[0]

print("1.Train Set Accuracy:\nAccuracy:", "{:.2%}".format(success/divisor))

print("\n2.10-Fold Cross-Validation Results:")
folds = fold10Sample(data)
successes = [0] * 10
for i in range(0,10):
    divisor = folds[i].shape[0]
    folds[i]['result'] = id3(data.drop(folds[i].index), folds[i])
    successes[i] = folds[i][folds[i]['result'] == folds[i]['Class']].shape[0]/divisor
    print("Accuracy Fold", i+1, ":", "{:.2%}".format(successes[i]))

print("\nAverage Accuracy:",  "{:.2%}".format(np.average(successes)))
print("Standard Deviation:",  "{:.2%}".format(np.std(successes)))

testing['result'] = id3(data, testing)
success = testing[testing['result'] == testing['Class']].shape[0]
divisor = testing.shape[0]
print("\n3.Test Set Accuracy:\nAccuracy:", "{:.2%}".format(success/divisor))



1.Train Set Accuracy:
Accuracy: 0.00%

2.10-Fold Cross-Validation Results:
Accuracy Fold 1 : 0.00%
Accuracy Fold 2 : 0.00%
Accuracy Fold 3 : 0.00%
Accuracy Fold 4 : 0.00%
Accuracy Fold 5 : 0.00%
Accuracy Fold 6 : 0.00%
Accuracy Fold 7 : 0.00%
Accuracy Fold 8 : 0.00%
Accuracy Fold 9 : 0.00%
Accuracy Fold 10 : 0.00%

Average Accuracy: 0.00%
Standard Deviation: 0.00%

3.Test Set Accuracy:
Accuracy: 0.00%
